In [1]:
import numpy as np
import math
from scipy.optimize import linprog
from abc import ABC

In [25]:
# Test out with random dataset 

def generate_bit_data(n = 1000): 
    return np.random.randint(0,2,n)


class Mechanism(ABC): 
    pass

class NoiseMechanism(Mechanism):
    def __init__(self, dataset):
        self.dataset = dataset
        self.length = len(dataset)    
        
    def get_len(self):
        return self.length
    
    def get_raw_data(self):
        return self.dataset


class LaplaceMechanism(NoiseMechanism):
    def __init__(self, dataset, mean = 0.0, scale = 1.0):
        super().__init__(dataset)
        self.mean = mean 
        self.scale = scale
    
    def query(self, indices): 
        result = np.sum(self.dataset[indices])
        noise = np.random.laplace(loc=self.mean, scale=self.scale, size=None)
        return result + noise

class GaussianMechanism(NoiseMechanism):
    def __init__(self, dataset, mean = 0.0, scale = 1.0):
        super().__init__(dataset)
        self.mean = mean 
        self.scale = scale
    
    def query(self, indices): 
        result = np.sum(self.dataset[indices])
        noise = np.random.normal(loc=self.mean, scale=self.scale, size=None)
        return result + noise

    
    
def dinur_nissim(mech, epsilon = 0.5, gen_t = lambda n : n * (math.log2(n) ** 2), should_log = False):
    n = mech.get_len()
    t = int(gen_t(n))

    def generate_query():
        # Generate a random subset of [n] by generating a list of bits and convert to indices
        q_j = np.random.randint(0,2,n)
        indices2 = q_j.nonzero()
        return (q_j, indices2) 
    
    # Make t queries to the mechanism. A_ub and b_ub are respectively the coefficient matrix 
    #   and upper bound for the linear program 
    A_ub = []
    b_ub = []
    for _ in range(int(t)):
        q, indices = generate_query()
        answer = mech.query(indices)
        A_ub.append(q)
        b_ub.append(answer + epsilon)
        A_ub.append(-1 * q)
        b_ub.append(epsilon - answer)
    A_ub = np.vstack(A_ub)
    b_ub = np.array(b_ub)
    # Rounding Phase: round the results of the LP into integers
    
    if(should_log):
        print(f"Solving LP with {len(b_ub)} constraints")
    program_result = linprog(np.zeros(n), np.vstack(A_ub), b_ub, bounds=(0,1))

    if(should_log):    
        print(f"Done w/ LP")
    round_vector = np.vectorize(round, otypes=[int])
    return round_vector(program_result["x"])

In [27]:
data = generate_bit_data(n=500)

mech = LaplaceMechanism(data, mean = 0, scale = 0)
dinur_nissim(mech, gen_t=lambda n : n)

array([0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0,
       0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1,
       1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1,
       0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0,
       1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0,

In [31]:
data = generate_bit_data(n=500)

def percent_reconstructed(data, results): 
    n = len(data)
    return 1 - ((np.count_nonzero(data!=results))/n)

for s in range(100):
    scale = s/10
    mech = GaussianMechanism(data, mean = 0, scale = scale)
    result = dinur_nissim(mech, gen_t=lambda n : n)
#     print(f"Data: {data}\n")
#     print(f"Result: {result}\n")
    print(f"Scale={scale}---% reconstructed={100*percent_reconstructed(data, result)}%")
    

Scale=0.0---% reconstructed=100.0%
Scale=0.1---% reconstructed=100.0%
Scale=0.2---% reconstructed=100.0%
Scale=0.3---% reconstructed=100.0%
Scale=0.4---% reconstructed=100.0%
Scale=0.5---% reconstructed=100.0%
Scale=0.6---% reconstructed=100.0%
Scale=0.7---% reconstructed=100.0%
Scale=0.8---% reconstructed=99.8%
Scale=0.9---% reconstructed=100.0%
Scale=1.0---% reconstructed=99.8%
Scale=1.1---% reconstructed=98.0%
Scale=1.2---% reconstructed=98.4%
Scale=1.3---% reconstructed=96.2%
Scale=1.4---% reconstructed=95.39999999999999%
Scale=1.5---% reconstructed=91.0%
Scale=1.6---% reconstructed=93.39999999999999%
Scale=1.7---% reconstructed=84.0%
Scale=1.8---% reconstructed=81.4%
Scale=1.9---% reconstructed=74.6%
Scale=2.0---% reconstructed=74.8%
Scale=2.1---% reconstructed=66.6%
Scale=2.2---% reconstructed=65.4%
Scale=2.3---% reconstructed=60.6%
Scale=2.4---% reconstructed=56.800000000000004%
Scale=2.5---% reconstructed=55.60000000000001%
Scale=2.6---% reconstructed=62.4%
Scale=2.7---% recons

KeyboardInterrupt: 

In [ ]:
data, result
